## Download the data

In [1]:
import os.path
if not os.path.isfile('data/vmware_ir_content.csv'):
    !pip install kaggle
    !kaggle competitions download -c vmware-zero-shot-information-retrieval
    !mkdir -p data/
    !unzip -o vmware-zero-shot-information-retrieval.zip
    !mv *.csv data/

## Parse the vmware corpus

- Load vmware text
- Read metadata into fields

In [40]:
import pandas as pd
corpus = pd.read_csv('data/vmware_ir_content.csv')
queries = pd.read_csv('data/test.csv')
corpus = corpus.fillna('')

import json
import ast

parsed_rows = []

for row in corpus.to_dict(orient='records'):
    row_dict = ast.literal_eval(row['raw_meta'])
    row_dict['id'] = row['f_name']
    row_dict['f_name'] = row['f_name']
    row_dict['raw_text'] = row['raw_text']
    row_dict['document_group'] = row['document_group']

    # fields that dont index for some reason
    bad_fields = ["DC.Date", "DC.Publisher"]
    for bad_field in bad_fields:
        if bad_field in row_dict:
            del row_dict[bad_field]
        
    parsed_rows.append(row_dict)
    
corpus = pd.DataFrame(parsed_rows)
corpus

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
0,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Black Duck for VMware Tanzu | Solutions Hub | ...,authenticity_token,NnLorKoXWVAXtOGcZgn6cD0qO7KbHyuGc6QwI0FaGybXtO...,Black Duck for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Percona | Solutions Hub | VMware Tanzu,authenticity_token,2N9nj_cHWOjsFUJe2rbR8Rlllj9DfNaD9m79W4GYB9IlnP...,Percona | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,United States Air Force | Customers | VMware ...,authenticity_token,CCmw9wy9YaWEur35yHQYIvF6XbSJ23oUmZsF2qf5Qmyo83...,United States Air Force | Customers,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Dynatrace for VMware Tanzu | Solutions Hub | V...,authenticity_token,VqAn3GNy9pSHWoDrlxi_mxbAM6XrBFTcIDQeYUxvWCehtX...,Dynatrace for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,New Relic for VMware Tanzu | Solutions Hub | V...,authenticity_token,VM0vueZ5tE_2BnTwD-f09O5A3bKGe-eCZO2ixNeHAzd-hW...,New Relic for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
#for row in corpus[corpus['id'].str.contains('GUID')]['id']:
#    print(row)

corpus

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
0,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Black Duck for VMware Tanzu | Solutions Hub | ...,authenticity_token,NnLorKoXWVAXtOGcZgn6cD0qO7KbHyuGc6QwI0FaGybXtO...,Black Duck for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Percona | Solutions Hub | VMware Tanzu,authenticity_token,2N9nj_cHWOjsFUJe2rbR8Rlllj9DfNaD9m79W4GYB9IlnP...,Percona | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,United States Air Force | Customers | VMware ...,authenticity_token,CCmw9wy9YaWEur35yHQYIvF6XbSJ23oUmZsF2qf5Qmyo83...,United States Air Force | Customers,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Dynatrace for VMware Tanzu | Solutions Hub | V...,authenticity_token,VqAn3GNy9pSHWoDrlxi_mxbAM6XrBFTcIDQeYUxvWCehtX...,Dynatrace for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,New Relic for VMware Tanzu | Solutions Hub | V...,authenticity_token,VM0vueZ5tE_2BnTwD-f09O5A3bKGe-eCZO2ixNeHAzd-hW...,New Relic for VMware Tanzu | Solutions Hub,"paas, platform as a service, agile, mobile, oss",EN,Hzjd9ksxUfQs-ti7CSAoFUmvHvcNHwYT-xtRRhvb5eA,8sc5peq98k3k11motigpxpzqkksya2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from ltr.client import ElasticClient
client=ElasticClient()

from ltr.index import rebuild
rebuild(client, index='vmware', doc_src=parsed_rows, force=True)

Deleted index vmware [Status: 200]
Created index vmware [Status: 200]
Streaming Bulk index DONE vmware [Status: 201]


In [66]:
def search(client, query):
    es = client.es
    body = {
        'size': 5,
        'query': {
            'multi_match': {
                'fields': ['title', 'description', 'raw_text'],
                'type': 'cross_fields',
                'query': query
            }
        }
    }
    return es.search(index='vmware', body=body)['hits']['hits']


def search_all(client, queries):
    all_results = []
    for query in queries.to_dict(orient='records'):
        print(f"Searching for {query['QueryId']},{query['Query']}")
        results = search(client, query['Query'])
        for rank, result in enumerate(results):
            source = result['_source']
            if rank == 0 and 'titleTag' in source:
                print(source['titleTag'])
            source['rank'] = rank
            source['DocumentId'] = source['id']
            source['QueryId'] = query['QueryId']
            all_results.append(source)
        print()
        print()
    all_results = pd.DataFrame(all_results)
    return queries.merge(all_results, how='left', on='QueryId').sort_values(['QueryId', 'rank'])
    
submission = search_all(client, queries)

Searching for 0,what is hypervisor
What is a Bare Metal Hypervisor? | VMware Glossary


Searching for 1,what is a hypervisor
What is a Bare Metal Hypervisor? | VMware Glossary


Searching for 2,what is vmware
What’s New in Horizon 8 (2012) | VMware


Searching for 3,unable to login because you do not have permission on any vcenter server systems connected to this client.
Troubleshooting permissions errors when connecting to vCenter Server with the vSphere Client (1003872) | VMware KB


Searching for 4,what is virtualization
What is virtualization management&how does it work ? | VMware


Searching for 5,what is a virtual machine
Can an admin peek inside my VM? - VMware vSphere Blog


Searching for 6,how to install vmware tools ubuntu
Installing VMware Tools in a Ubuntu virtual machine - VMware Support Insider


Searching for 7,how to use vmware


Searching for 8,how to install vmware tools


Searching for 9,how to install vmware tools linux


Searching for 10,unable to login because you

What is a Bare Metal Hypervisor? | VMware Glossary


Searching for 73,what is virtual network
What is Network Functions Virtualization (NFV)? | VMware Glossary


Searching for 74,how to update esxi host


Searching for 75,what is workspace one


Searching for 76,what is scalability in cloud computing
What is Cloud Scalability? | VMware Glossary


Searching for 77,how to install vmware tools in linux


Searching for 78,how to ping in mac
Testing network connectivity with the ping command on a Mac (1020983) | VMware KB


Searching for 79,how to make a virtual machine


Searching for 80,how to create a virtual machine in vmware


Searching for 81,what is a virtual network
What is Network Functions Virtualization (NFV)? | VMware Glossary


Searching for 82,what is virtual memory ? with the help of neat sketch explain the method of virtual to physical address translation.
How is Virtual Memory Translated to Physical Memory? - VMware vSphere Blog


Searching for 83,the redo log of vmdk is co



Searching for 157,how to convert thick to thin provisioned in vmware
Storage vMotion or migration fails to convert disk from thick provisioned to thin provisioned when moving within same datastore (1026829) | VMware KB


Searching for 158,how to check esxi version


Searching for 159,what is hypervisor?
What is a Bare Metal Hypervisor? | VMware Glossary


Searching for 160,how to find vcenter version
Workspace ONE UEM: Build a Zero Touch USB with Encrypted PPKG - Deep Dive | VMware


Searching for 161,what is enterprise security
How KRACK Affects Your Mobile Workforce&What You Can Do About It | VMware End-User Computing Blog


Searching for 162,what is pod in kubernetes
What are Kubernetes Pods? | VMware Glossary


Searching for 163,unable to login because you do not have permission on any vcenter server systems connected to this client. back to login screen
Troubleshooting permissions errors when connecting to vCenter Server with the vSphere Client (1003872) | VMware KB


Searching 

How to move ESXi/ESX host from one vCenter Server to another (1004775) | VMware KB


Searching for 228,how to reset forgotten vmware esxi 6.7 root password
Resetting root password in vCenter Server Appliance 6.5 / 6.7 / 7.x (2147144) | VMware KB


Searching for 229,how to check powercli version
Updating PowerCLI through the PowerShell Gallery - VMware PowerCLI Blog


Searching for 230,what is vrealize
What’s New with vRealize Automation 8.3 – Technical Overview - VMware Cloud Management


Searching for 231,how does vdi work
[Video] How Does Samsung DeX Work With VMware Workspace ONE? | VMware End-User Computing Blog


Searching for 232,how to open shell in windows
Opening a command or shell prompt (1003892) | VMware KB


Searching for 233,what is daas


Searching for 234,what tasks do hypervisors accomplish


Searching for 235,how to install vmware tools on windows


Searching for 236,how to download vsphere client


Searching for 237,how to upgrade vmfs 5 to 6
Upgrading VMware ESXi 4.



Searching for 305,what is vcenter server


Searching for 306,what is virtual machine security
Can an admin peek inside my VM? - VMware vSphere Blog


Searching for 307,how to import ovf file in vmware


Searching for 308,what is vmkernel


Searching for 309,how to change vcenter ip address


Searching for 310,how to make virtual machine


Searching for 311,check if iis is running


Searching for 312,how to install vmware tools centos 7
VMware Tools installation or upgrade might fail due to incorrect selection of VMware Tools ISO image (2147454) | VMware KB


Searching for 313,how to copy virtual machine to another computer


Searching for 314,how to connect vcenter using powercli


Searching for 315,how to install windows 10 vmware
Windows 10 Modern Management | VMware


Searching for 316,what is general prerequisite of hardware clustering


Searching for 317,how to create a new virtual machine in vmware


Searching for 318,what does disaster recovery meaning
What is Disaster Recover

...

How to unenroll,  reconfigure, and re-enroll DEP devices (80539) | VMware KB


Searching for 2017,how much ram does vmware need
In Horizon View 5.3 VMware Horizon HTML Access, manually changing the remote desktop's screen resolution to 2560x1920 causes a disconnection (2063313) | VMware KB


Searching for 2018,what is disaster recovery plan
What is Disaster Recovery? | VMware Glossary


Searching for 2019,how do vms work


Searching for 2020,what is meant by cloud native
What is Telco Cloud Infrastructure | VMware


Searching for 2021,how to mount datastore esxi 6.7


Searching for 2022,what is vmware vsphere?
KBTV Webinars - SSL certificate handling in VMware vSphere 6 - VMware Support Insider


Searching for 2023,how to join esxi host to domain
ESXi 6.0 hosts become unresponsive when joined to an AD domain (2145611) | VMware KB


Searching for 2024,how to install windows 10 on vmware workstation 15 pro
Upgrading from VMware Workstation Pro 14 to VMware Workstation Pro 15 (2126729) | 

Resetting root password in vCenter Server Appliance 6.5 / 6.7 / 7.x (2147144) | VMware KB


Searching for 2096,how to fix the system administrator has set policies to prevent this installation in windows 10
"The system administrator has set policies to prevent this installation" error when installing VMware Tools on a Windows Guest (1016563) | VMware KB


Searching for 2097,how to click ctrl alt delete in vmware


Searching for 2098,what is the difference between soa (service oriented architecture) and cloud computing
SOA and Cloud Computing: Are They The Same? - VMware Cloud Provider Blog


Searching for 2099,how to remove datastore from vcenter


Searching for 2100,how to increase hard disk space in vmware vsphere 6
vRealize Automation appliance runs out of disk space (2133183) | VMware KB


Searching for 2101,how to set default camera on mac


Searching for 2102,what is cold migration in vmware


Searching for 2103,what is vmware carbon black


Searching for 2104,how to create ova f

"Cannot download software packages from patch source. Check the events and the Update Manager log for download details" error when remediating the ESXi host using VMware Update Manager (64995) | VMware KB


Searching for 2171,how to take vmware backup
How to take backup for Image from Horizon Cloud Admin Portal (56902) | VMware KB


Searching for 2172,how to change screen resolution in redhat linux 7
Using a Screen Resolution of 1024 x 768 in a Red Hat Enterprise Linux 5 Virtual Machine (1001511) | VMware KB


Searching for 2173,what is hyper vision
Destination VMworld - Professional Development Track - VMworld Blog


Searching for 2174,how to change vcenter administrator password


Searching for 2175,how to check if ssh server is running
How to check vCenter Server essential service status and dependencies (2041357) | VMware KB


Searching for 2176,how to upgrade vmware 6.0 to 6.7
License key requirements for new version of VMware products (2059926) | VMware KB


Searching for 2177,ho

How to import an Oracle VirtualBox virtual machine into VMware Fusion, Workstation or Player - VMware Support Insider


Searching for 2257,what is virtualization security
What is Virtualized Security? | VMware Glossary


Searching for 2258,how to check snapshot size in vmware
Consolidating snapshots in VMware Fusion (1020000) | VMware KB


Searching for 2259,how to install vmware workstation pro on windows 10
Workstation Player Community


Searching for 2260,how to install linux vmware


Searching for 2261,what is deduplication and compression
vSAN Health Service - Cluster Health - Deduplication and compression configuration consistency check (2144279) | VMware KB


Searching for 2262,what is an ovf template


Searching for 2263,what is application modernization
What is Application Modernization? | VMware Glossary


Searching for 2264,what is v center


Searching for 2265,what are the automation level on a drs cluster


Searching for 2266,how to check ping in terminal mac
Adding Airpri

How can backup software read version 3 virtual disk? (2064959) | VMware KB


Searching for 2349,how to open vmware workstation


Searching for 2350,kubernetes what is a namespace
What is a Kubernetes Namespace? | VMware Glossary




In [70]:
def write_submission():
    from time import time
    timestamp = str(time()).replace('.', '')
    fname = f'data/turnbull_submission_{timestamp}.csv'
    print("Writing To: ", fname)
    submission[['QueryId', 'DocumentId']].to_csv(fname, index=False)

In [71]:
submission[['QueryId', 'DocumentId']]

,QueryId,DocumentId
0,0,https---www.vmware.com-topics-glossary-content...
1,0,https---www.vmware.com-products-vsphere-hyperv...
2,0,https---www.vmware.com-topics-glossary-content...
3,0,https---www.vmware.com-topics-glossary-content...
4,0,https---www.vmware.com-topics-glossary-content...
...,...,...
11750,2350,https---www.vmware.com-topics-glossary-content...
11751,2350,en-2021-vRealize-Automation-8-3-Using-and-Mana...
11752,2350,en-2021-vRealize-Automation-8-4-Using-and-Mana...
11753,2350,en-2021-VMware-Cloud-Assembly-services-Using-a...


In [72]:
write_submission()

Writing To:  data/turnbull_submission_1652544680901428.csv


In [57]:
corpus[corpus['id'].str.contains('https---blogs.vmware.com-cloudprovider-2015-11-simplifying-cloud-spending-with-vmware-subscription-purchase-program.txt')]

,onetrust-data-domain,microsites-utag,titleTag,csrf-param,csrf-token,DC.Title,DC.Subject,DC.Language,google-site-verification,facebook-domain-verification,...,machine-translated,name,custom_label,link_url,children,url,lastModified,md5,publication-author-name,output-file-name
11041,b9242434-9feb-47be-a894-3a9e658fdd50,https://tags.tiqcdn.com/utag/vmware/microsites...,Simplifying Cloud Spending with VMware Subscri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
